In [61]:
# We import dataclass to represent 1 gene
from dataclasses import dataclass
import random
import pandas as pd
import numpy as np
from deap import base, creator, tools, algorithms

In [62]:
# A gene contains 2 attributes : the food item's name and the number of servings of this item
@dataclass
class Gene:
    food_item_name: str
    serving: int

In [63]:
# We make a list representing all the meals we have in 1 chromosome 
Meals =["Breakfast", "Snack1", "Lunch", "Snack2", "Dinner"]

# We make a list representing all the food groups we have in 1 meal
Food_Groups = ["Protein", "Carbs", "Fats", "Vegetables", "Fruits", "Dairy"]

# Take the info from the dataset 
Dataset = pd.read_csv(r"/Users/novi/Desktop/Bahrain Food Dataset.csv")


In [64]:
# I created a dictionary that contains all the saudi guideline's servings
SERVING_GUIDELINES_BY_KCAL = {
    1200: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Lunch":     {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 2, "Fruits": 0, "Fats": 0},
        "Snack2":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats": 0}
    },

    1300: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack2":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats": 0}
    },

    1400: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 1, "Vegetables": 2, "Fruits": 0, "Fats": 1},
        "Snack2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats": 0}
    },

    1500: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats": 0}
    },

    1600: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats": 0}
    },

    1700: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 2, "Fruits": 0, "Fats": 1},
        "Snack2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats": 0}
    },

    1800: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats": 1}
    },

    1900: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 2, "Fruits": 0, "Fats": 1},
        "Snack2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats": 1}
    },

    2000: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack2":    {"Grains": 2, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 2, "Vegetables": 1, "Fruits": 1, "Fats": 0}
    },

    2200: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 1, "Fruits": 0, "Fats": 1},
        "Snack1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 2, "Fruits": 0, "Fats": 1},
        "Snack2":    {"Grains": 2, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 2, "Vegetables": 1, "Fruits": 1, "Fats": 1}
    }
}

In [65]:
import pandas as pd

CSV_PATH = r"/Users/novi/Desktop/Bahrain Food Dataset.csv"
df = pd.read_csv(CSV_PATH)

df["Food Item"] = df["Food Item"].astype(str).str.strip()
df["Category"] = df["Category"].astype(str).str.strip()

num_cols = ["Protein (g)", "Fats (g)", "Carbohydrates (g)", "Calories (kcal)"]
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df = df.dropna(subset=num_cols)

# ✅ CSV categories -> code groups
CATEGORY_ALIAS = {
    "Grains": "Carbs",
    "Fats and Oils": "Fats",
    "Protein": "Protein",
    "Vegetables": "Vegetables",
    "Fruits": "Fruits",
    "Dairy": "Dairy",
}

FOOD_DATA = {g: {} for g in set(CATEGORY_ALIAS.values())}

for _, row in df.iterrows():
    cat = row["Category"]
    if cat not in CATEGORY_ALIAS:
        continue
    group = CATEGORY_ALIAS[cat]   # ✅ Grains -> Carbs
    name = row["Food Item"]

    FOOD_DATA[group][name] = {
        "p": float(row["Protein (g)"]),
        "f": float(row["Fats (g)"]),
        "c": float(row["Carbohydrates (g)"]),
        "cal": float(row["Calories (kcal)"]),
    }

print("FOOD_DATA keys:", FOOD_DATA.keys())
print("Carbs count:", len(FOOD_DATA["Carbs"]))


FOOD_DATA keys: dict_keys(['Protein', 'Vegetables', 'Carbs', 'Dairy', 'Fats', 'Fruits'])
Carbs count: 40


In [ ]:
# DEAP Configuration
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
SERVING_GUIDELINES = SERVING_GUIDELINES_BY_KCAL[1600]
g = "Grains" if group == "Carbs" else group
serving_count = SERVING_GUIDELINES_BY_KCAL[TARGET_KCAL][meal].get(g, 0)

def create_individual():
    genes = []
    for meal in Meals:
        for group in Food_Groups:
            g = "Grains" if group == "Carbs" else group
            serving_count = SERVING_GUIDELINES_BY_KCAL[TARGET_KCAL][meal].get(g, 0)

            food_name = random.choice(list(FOOD_DATA[group].keys()))  # group is Carbs here ✅
            genes.append(Gene(food_name, serving_count))
    return creator.Individual(genes)


toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Mutation
def mutate_meal_plan(individual):
    mutations_done = 0
    all_indices = list(range(len(individual))) # List of all possible indices in the chromosome
    random.shuffle(all_indices) # Shuffle to pick randomly without repeating the same one

    for i in all_indices:
        if mutations_done >= 2:
            break  # Stop once we have mutated 2 genes

        # Check if the serving size is not zero
        if individual[i].serving > 0:
            # Find which group it belongs to (Vegetables, Grains, ...)
            group_name = Food_Groups[i % 6]
            # Pick a new food name from food group dictionary, but the serving remains the same
            individual[i].food_item_name = random.choice(list(FOOD_DATA[group_name].keys()))
            mutations_done += 1
    return individual,


def evaluate_meal_plan(individual):
    total_p = 0.0 # Protein grams
    total_f = 0.0 # Fat grams
    total_c = 0.0 # Carbohydrates grams

    # First: Sum up all grams based on food item and serving count
    for gene in individual:
        group_name = None
        # Find which group this food belongs to
        for group, foods in FOOD_DATA.items():
            if gene.food_item_name in foods:
                group_name = group
                break

        if group_name:
            stats = FOOD_DATA[group_name][gene.food_item_name]
            total_p += stats['p'] * gene.serving
            total_f += stats['f'] * gene.serving
            total_c += stats['c'] * gene.serving

    # Second : Calculate actual macro percentages (p_c, p_p, p_f)
    g_total = total_p + total_f + total_c
    if g_total == 0: return 0, # Avoid division by zero

    p_p = total_p / g_total
    p_f = total_f / g_total
    p_c = total_c / g_total

    # Third :Target Ratios (The AMDR values)
    r_p, r_f, r_c = 0.20, 0.25, 0.55
    # Fourth: Calculate J_macro (Squared Error)
    j_macro = (p_c - r_c)**2 + (p_p - r_p)**2 + (p_f - r_f)**2

    # Fifth: apply the variety constraint
    N = 30 # total number of food items in the chromosome ( N is taken from the report)
    # count food appearance
    food_appearance={}
    for gene in individual:
        food_appearance[gene.food_item_name]= food_appearance.get(gene.food_item_name, 0) + 1

    # Calculate the penalty for variety
    j_var = 0
    for food_name,count in food_appearance.items():
        # we penalize if the food item appears more than once
        if count > 1:
            j_var += max(0,count-1)**2
    # Normalize the penalty (revise it )
    j_var = j_var / ((N-1)**2)

    # Sixth: calculate the score
    score = j_macro + j_var
    # store the penalties to print later
    individual.j_var= j_var
    individual.j_macro = j_macro

    # Lastly: We want to minimize score, so we return a fitness that increases as score decreases
    return 1 / (1 + score),


toolbox.register("evaluate", evaluate_meal_plan)
toolbox.register("mate", tools.cxTwoPoint) # One/Two-point meal-level crossover
toolbox.register("select", tools.selTournament, tournsize=3) # Tournament selection
toolbox.register("mutate", mutate_meal_plan)

# Execution Loop
def run_simulation(target_kcal):
    global TARGET_KCAL
    TARGET_KCAL = target_kcal   # tells create_individual which guideline to use

    pop = toolbox.population(n=50) # Population = 50

    hof = tools.HallOfFame(1) # Keeps the absolute best found across all generations

    mu = len(pop) # Number of individuals to select for the next generation
    lambd = 100 # Number of children to produce in each generation

    # Statistics object
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Run Elitism-Based GA
    algorithms.eaMuPlusLambda(pop, toolbox,
                               mu=mu,
                               lambda_=lambd,
                               cxpb=0.7,   # Crossover probability
                               mutpb=0.2,  # Mutation probability
                               ngen=20,    # Number of generations
                               stats=stats,
                               halloffame=hof,
                               verbose=True)

    return hof[0]

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [67]:
# Execute the Simulation
best_plan = run_simulation(1600)

# Calculate Final Nutritional Totals
total_p, total_f, total_c, total_cal = 0.0, 0.0, 0.0, 0.0

print(f"\n{'MEAL':<12} | {'GROUP':<10} | {'FOOD ITEM':<18} | {'SERV.':<6} | {'P (g)':<6} | {'F (g)':<6} | {'C (g)':<6}")
print("-" * 75)

for i, gene in enumerate(best_plan):
    meal = Meals[i // 6]
    group = Food_Groups[i % 6]

    # Get food stats from dataset
    stats = FOOD_DATA[group][gene.food_item_name]

    # Calculate totals for this gene
    p = stats['p'] * gene.serving
    f = stats['f'] * gene.serving
    c = stats['c'] * gene.serving
    cal = stats['cal'] * gene.serving

    # Update running totals
    total_p += p
    total_f += f
    total_c += c
    total_cal += cal

    # Print row (only if servings > 0 for clarity)
    if gene.serving > 0:
        print(f"{meal:<12} | {group:<10} | {gene.food_item_name:<18} | {gene.serving:<6} | {p:<6.1f} | {f:<6.1f} | {c:<6.1f}")

# Calculate Final Percentages
g_total = total_p + total_f + total_c
p_p = (total_p / g_total) * 100
p_f = (total_f / g_total) * 100
p_c = (total_c / g_total) * 100
variety= (1-best_plan.j_var)*100

# Print Summary Report
print("\n" + "="*40)
print("       DAILY NUTRITION SUMMARY")
print("="*40)
print(f"Total Energy:   {total_cal:.1f} kcal")
print(f"Total Weight:   {g_total:.1f} g (Macro Grams)")
print("-" * 40)
print(f"Nutrient     | Actual % | Target %")
print(f"Carbs (c)    | {p_c:>7.1f}% | 55.0%")
print(f"Protein (p)  | {p_p:>7.1f}% | 20.0%")
print(f"Fats (f)     | {p_f:>7.1f}% | 25.0%")
print(f"Meal variety | {variety:>7.1f}% | 100%")
print("="*40)

gen	nevals	avg     	min     	max     
0  	50    	0.966783	0.926317	0.995407
1  	95    	0.981768	0.957384	0.995407
2  	92    	0.987953	0.975952	0.996413
3  	90    	0.993483	0.98608 	0.996413
4  	86    	0.994547	0.988266	0.996413
5  	89    	0.995528	0.990861	0.99863 
6  	88    	0.996269	0.993323	0.999206
7  	88    	0.997521	0.993323	0.999439
8  	93    	0.998499	0.996262	0.999439
9  	90    	0.998964	0.995661	0.999822
10 	90    	0.999253	0.997944	0.999822
11 	87    	0.999333	0.996741	0.999822
12 	87    	0.999465	0.999206	0.999822
13 	92    	0.99958 	0.999206	0.999916
14 	83    	0.999682	0.999383	0.999916
15 	94    	0.9998  	0.999439	0.99999 
16 	88    	0.999851	0.99807 	0.99999 
17 	91    	0.999933	0.999631	0.99999 
18 	93    	0.999969	0.999765	0.999996
19 	91    	0.999987	0.999962	0.999996
20 	89    	0.999991	0.99999 	0.999996

MEAL         | GROUP      | FOOD ITEM          | SERV.  | P (g)  | F (g)  | C (g) 
---------------------------------------------------------------------------
Brea